In [5]:
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
from mindquantum.core.gates import X
from mindquantum.core.circuit import Circuit
from mindquantum.core.operators import Hamiltonian
from mindquantum.simulator import Simulator
from mindquantum.algorithm.nisq import generate_uccsd
import mindspore as ms

ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
dist = 1.0
geometry = [
    ["H", [0.0, 0.0, 0.0 * dist]],
    ["H", [0.0, 0.0, 1.0 * dist]],
]
basis = "sto3g"
spin = 0
print("Geometry: \n", geometry)
molecule_of = MolecularData(
    geometry,
    basis,
    multiplicity=2 * spin + 1
)
molecule_of = run_pyscf(
    molecule_of,
    run_scf=1,
    run_ccsd=1,
    run_fci=1
)
print("FCI energy: %20.16f Ha" % (molecule_of.fci_energy))
hartreefock_wfn_circuit = Circuit([X.on(i) for i in range(molecule_of.n_electrons)])
ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, n_electrons = generate_uccsd(molecule_of, threshold=-1)


Geometry: 
 [['H', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 1.0]]]
FCI energy:  -1.1011503302326187 Ha
ccsd:-1.1011503302444787.
fci:-1.1011503302326187.


In [6]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

         Circuit Summary         
╭──────────────────────┬────────╮
│ Info                 │ value  │
├──────────────────────┼────────┤
│ Number of qubit      │ 4      │
├──────────────────────┼────────┤
│ Total number of gate │ 158    │
│ Barrier              │ 48     │
│ Noise Channel        │ 0      │
│ Measurement          │ 0      │
├──────────────────────┼────────┤
│ Parameter gate       │ 12     │
│ 2 ansatz parameters  │ p0, p1 │
╰──────────────────────┴────────╯

Number of parameters: 2
